<a href="https://colab.research.google.com/github/KGL8/Code-for-Lab/blob/main/photon_through_deuteron_target.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pythreejs

import numpy as np
from pythreejs import *
import ipywidgets as widgets

In [10]:
def vector_from_angles(theta, phi):
    return np.array([np.sin(theta), np.cos(theta) * np.sin(phi), np.cos(theta) * np.cos(phi)])

def intersect_ray_box(ray_origin, ray_direction, box_min, box_max):
    pass

focal_point_distance = 3
width = 2
height = 5
depth = 7
box_min = np.array([-width / 2, -height / 2, focal_point_distance])
box_max = np.array([width / 2, height / 2, depth + focal_point_distance])
ray_direction = vector_from_angles(0.5,0.5)

In [15]:
def ray(origin, direction, length, col):
    return Line(geometry=BufferGeometry(attributes={'position': BufferAttribute(array=np.array([origin, length * np.array(direction)]), normalized=False),}),material=LineBasicMaterial(color=col))

In [22]:
def point(position, col, size):
    return Points(geometry=BufferGeometry(attributes={"position": BufferAttribute(np.array([position]), normalized=False)}),
                  material=PointsMaterial(color=col, size=size, sizeAttenuation=False))

In [29]:
def box(width, height, depth, col):
  box_geometry = BoxBufferGeometry(width=width, height=height, depth=depth)
  box_material = MeshBasicMaterial(color=col, wireframe=True)
  return Mesh(geometry=box_geometry, material=box_material)

The right triangle that defines the distance between the origin and each corner of rectangular prism is leg1 = focal point distance, leg2 = sqrt(1/4(hight^2+width^2)), hypotenuse = you can figure it out.

the only trouble now is converting the angle to spherical coordinates

In [37]:
corner = np.array([width/2, height/2, focal_point_distance])


In [42]:
target_box = box(width, height, depth, 'red')
target_box.position = [0, 0, depth / 2 + focal_point_distance]

ray_line = ray(np.array([0, 0, 0]), ray_direction, 100, '#f5f542')

z_ray = ray(np.array([0, 0, 0]), vector_from_angles(0, 0), 1, 'red')
y_ray = ray(np.array([0, 0, 0]), vector_from_angles(0, np.pi/2), 1, 'green')
x_ray = ray(np.array([0, 0, 0]), vector_from_angles(np.pi/2, 0), 1, 'blue')

origin_point = point([0, 0, 0], '#ffffff', 4)

ambient_light = AmbientLight(color='white', intensity=0.2)

scene = Scene(children=[target_box, ray_line, origin_point, ambient_light, x_ray, y_ray, z_ray])
scene.background = '#141414'
camera = PerspectiveCamera(position=[10, 10, 10], up=[0, 1, 0], children=[DirectionalLight(color='white', position=[3, 5, 1], intensity=0.5)])

display(Renderer(camera=camera, scene=scene, controls=[OrbitControls(controlling=camera)], width=800, height=600))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_539e0fe3-280e-4ef8-9b9d-7829a08348ee'.

In [1]:
from google.colab import output
output.enable_custom_widget_manager()